<a href="https://colab.research.google.com/github/kurakanja/113-2coding/blob/main/%E5%A4%A7%E5%AD%B8%E5%AD%B8%E7%B3%BB%E8%A8%8E%E8%AB%96%E7%95%99%E8%A8%80%E6%96%87%E5%AD%97%E5%88%86%E9%A1%9E.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

此程式Colab連結:https://colab.research.google.com/drive/11-0TMK-XUslRKvDkVrrLx2EM6CvVWrP8?usp=sharing

大學學系討論:
https://docs.google.com/spreadsheets/d/1BM8FO5xeDox0Grfhy7I9YagYLIldxXvieH4gsB9fUFI/edit?usp=sharing

In [ ]:
from google.colab import auth
auth.authenticate_user()

import gspread
from google.auth import default
creds, _ = default()

gc = gspread.authorize(creds)

In [22]:
import pandas as pd
# read data and put it in a dataframe
# 在 google 工作表載入 gsheets
gsheets = gc.open_by_url('https://docs.google.com/spreadsheets/d/1BM8FO5xeDox0Grfhy7I9YagYLIldxXvieH4gsB9fUFI/edit?usp=sharing').sheet1

In [23]:
dicts = gc.open_by_url('https://docs.google.com/spreadsheets/d/1BM8FO5xeDox0Grfhy7I9YagYLIldxXvieH4gsB9fUFI/edit?usp=sharing').get_worksheet(0)
dicts = dicts.get_all_records()
dicts = pd.DataFrame(dicts)

In [24]:
# 讀取所有數據
rows = gsheets.get_all_records()
import pandas as pd
df = pd.DataFrame(rows)

In [25]:
df.head()

,時間戳記,姓名,校名,系所,留言
0,2025/4/24 上午 9:51:57,曾凱筠,東吳大學,哲學系,東吳哲學 學費+住宿接近五萬\n哲學不是文科其實你要有部分的理科邏輯基礎才有辦法一開始就適應...
1,2025/4/24 上午 9:53:51,Yoou Boonsin,真理大學,台灣文學系,真理台灣文學(後段私校有人選ㄇ)\n地理位置在新北市淡水區，公車班次沒有淡江來得多，接近鄰校...
2,2025/4/24 上午 9:55:39,微波小莫,元智大學,藝術及設計學系,"""學費+學雜沒記錯也快5萬\n師資不差 老師都蠻厲害的\n但我個人認為要很有興趣再考慮會比較..."
3,2025/4/24 上午 9:56:08,李豫慈,國立政治大學,歐洲語文學系德文組,"""政大位於台北市的邊陲，學校唯一一班直達市政府的公車每個星期五下午都是塞爆的狀態（是人擠到門..."
4,2025/4/24 上午 9:56:35,Ting I Yun,國立清華大學,人文社會學系,"""如果你對人文社會性別等主題或是創作有興趣\n卻不知道自己該鑽研哪個部分\n這裡算是個讓你有..."


In [26]:
!pip install jieba
!pip install snownlp

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 37.6/37.6 MB 51.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for snownlp: filename=snownlp-0.12.3-py3-none-any.whl size=37760946 sha256=228db15fb347887ff7262af8455e798b36402ec9e1763e0756609d4832a5b8c8
  Stored in directory: /root/.cache/pip/wheels/4a/fc/04/d1937c02b2a445b34754da55f71612a3df648a38d711bd17eb
Successfully built snownlp


In [29]:
import jieba
from snownlp import SnowNLP
import pandas as pd
from collections import Counter
import re
import plotly.express as px

In [30]:
for _, row in dicts.iterrows():
    word = row['留言']
    jieba.add_word(word, freq=10000)
    jieba.suggest_freq(word, tune=True)

Building prefix dict from the default dictionary ...
DEBUG:jieba:Building prefix dict from the default dictionary ...
Dumping model to file cache /tmp/jieba.cache
DEBUG:jieba:Dumping model to file cache /tmp/jieba.cache
Loading model cost 3.591 seconds.
DEBUG:jieba:Loading model cost 3.591 seconds.
Prefix dict has been built successfully.
DEBUG:jieba:Prefix dict has been built successfully.


In [31]:
# 只保留中文字
def extract_chinese(text):
    return ''.join(re.findall(r'[\u4e00-\u9fa5]', text))

In [32]:
df['Chinese_Content'] = df['留言'].apply(extract_chinese)

# 使用 Jieba 斷詞
df['Tokenized'] = df['Chinese_Content'].apply(lambda x: list(jieba.cut(x, HMM=True)))

# 展平成所有詞語的列表
all_words = [word for tokens in df['Tokenized'] for word in tokens if len(word) > 1]

In [ ]:
df['Tokenized']

,Tokenized
0,"[東吳, 哲學學費, 住宿, 接近, 五萬, 哲學, 不是, 文科, 其實, 你, 要, 有..."
1,"[真理, 台灣, 文學後段, 私校, 有人, 選, 地理位置, 在, 新北市, 淡水, 區公..."
2,"[學費, 學雜, 沒, 記錯, 也, 快, 萬師, 資不差, 老師, 都, 蠻, 厲害, 的..."
3,"[政, 大位, 於, 台北市, 的, 邊陲, 學校, 唯一, 一班, 直達, 市政府, 的,..."
4,"[如果, 你, 對, 人文, 社會, 性別, 等, 主題, 或是, 創作, 有, 興趣, 卻..."
5,"[雖然, 身為, 大一還, 沒太多, 經歷, 但, 就, 我, 所, 能, 的, 說, 乾,..."
6,"[學費, 部分, 我, 記得, 是繳, 差不多, 萬多系, 上, 內容, 的, 話, 有, ..."
7,"[學費, 第一期, 會, 比, 較, 多, 衣服, 那些, 的, 大概, 萬之後都, 是, ..."
8,"[學雜, 宿舍, 大約, 萬, 宿舍, 看住, 哪個, 宿舍, 價錢, 不, 一樣, 房型,..."
9,"[看到, 別人, 分享, 這串, 於, 是, 跑, 來, 這裡, 留言, 中正, 電機, 學..."


In [33]:
#下載東西
!apt-get update -qq
!apt-get install -y fonts-wqy-zenhei

W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following NEW packages will be installed:
  fonts-wqy-zenhei
0 upgraded, 1 newly installed, 0 to remove and 36 not upgraded.
Need to get 7,472 kB of archives.
After this operation, 16.8 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/universe amd64 fonts-wqy-zenhei all 0.9.45-8 [7,472 kB]
Fetched 7,472 kB in 2s (3,287 kB/s)
Selecting previously unselected package fonts-wqy-zenhei.
(Reading database ... 126333 files and directories currently installed.)
Preparing to unpack .../fonts-wqy-zenhei_0.9.45-8_all.deb ...
Unpacking fonts-wqy-zenhei (0.9.45-8) ...
Setting up fonts-wqy-zenhei (0.9.45-8) ...
Processing triggers for fontconfig (2.13.1-4.2ubuntu5) ...


In [36]:
from wordcloud import WordCloud

word_counts = Counter(all_words)
# 排除停用詞（這裡提供一個簡單的停用詞列表，可根據需要擴展）
stop_words = set(['所以', '好', '因為', '的', '是', '了', '我', '也', '在', '和', '就', '不', '有', '他', '她', '你','老師','我們','就是','很多','學校','不會','不是','真的','可以','自己','課程','部分','沒有','但是','基本上','一個','大概','這個','不用','除了','或是','覺得','之類','非常'])
filtered_word_counts = {word: count for word, count in word_counts.items() if word not in stop_words}
# 取前 300 個最常出現的詞
word_freq_df = pd.DataFrame(filtered_word_counts.items(), columns=['Word', 'Frequency'])
top_words = word_freq_df.head(300)

# 使用 WQY Zenhei 字體
wordcloud = WordCloud(
    width=800,
    height=400,
    background_color='white',
    font_path='/usr/share/fonts/truetype/wqy/wqy-zenhei.ttc'
)

# 根據詞頻生成文字雲
wordcloud.generate_from_frequencies(dict(zip(top_words['Word'], top_words['Frequency'])))

# 將文字雲轉換為圖像
wordcloud_img = wordcloud.to_array()

# 使用 Plotly 顯示文字雲
fig = px.imshow(wordcloud_img)
fig.update_layout(
    title="文字雲",
    xaxis=dict(showgrid=False, showticklabels=False, zeroline=False),
    yaxis=dict(showgrid=False, showticklabels=False, zeroline=False),
    coloraxis_showscale=False
)
fig.show()

In [37]:
import re
import pandas as pd


# 定義學校提取函數
def extract_school(校名):
    # 特例處理：如果提到特定學校，直接匹配
    if '藝術大學' in 校名:
        return '科技大學'
    elif '科技大學' in 校名:
        return '科技大學'
    else:
        return '一般大學'

# 提取學校名稱
df['學校類別'] = df['校名'].apply(lambda x: extract_school(str(x)))

# 應用函數到數據
df['學校類別'] = df['校名'].apply(lambda x: extract_school(str(x)))

In [38]:
df['學校類別']

,學校類別
0,一般大學
1,一般大學
2,一般大學
3,一般大學
4,一般大學
5,科技大學
6,科技大學
7,科技大學
8,科技大學
9,一般大學


In [39]:
df.head()

,時間戳記,姓名,校名,系所,留言,Chinese_Content,Tokenized,學校類別
0,2025/4/24 上午 9:51:57,曾凱筠,東吳大學,哲學系,東吳哲學 學費+住宿接近五萬\n哲學不是文科其實你要有部分的理科邏輯基礎才有辦法一開始就適應...,東吳哲學學費住宿接近五萬哲學不是文科其實你要有部分的理科邏輯基礎才有辦法一開始就適應我們學到...,"[東吳, 哲學學費, 住宿, 接近, 五萬, 哲學, 不是, 文科, 其實, 你, 要, 有...",一般大學
1,2025/4/24 上午 9:53:51,Yoou Boonsin,真理大學,台灣文學系,真理台灣文學(後段私校有人選ㄇ)\n地理位置在新北市淡水區，公車班次沒有淡江來得多，接近鄰校...,真理台灣文學後段私校有人選地理位置在新北市淡水區公車班次沒有淡江來得多接近鄰校走路就能到淡江...,"[真理, 台灣, 文學後段, 私校, 有人, 選, 地理位置, 在, 新北市, 淡水, 區公...",一般大學
2,2025/4/24 上午 9:55:39,微波小莫,元智大學,藝術及設計學系,"""學費+學雜沒記錯也快5萬\n師資不差 老師都蠻厲害的\n但我個人認為要很有興趣再考慮會比較...",學費學雜沒記錯也快萬師資不差老師都蠻厲害的但我個人認為要很有興趣再考慮會比較好常常做大型裝置...,"[學費, 學雜, 沒, 記錯, 也, 快, 萬師, 資不差, 老師, 都, 蠻, 厲害, 的...",一般大學
3,2025/4/24 上午 9:56:08,李豫慈,國立政治大學,歐洲語文學系德文組,"""政大位於台北市的邊陲，學校唯一一班直達市政府的公車每個星期五下午都是塞爆的狀態（是人擠到門...",政大位於台北市的邊陲學校唯一一班直達市政府的公車每個星期五下午都是塞爆的狀態是人擠到門邊的那...,"[政, 大位, 於, 台北市, 的, 邊陲, 學校, 唯一, 一班, 直達, 市政府, 的,...",一般大學
4,2025/4/24 上午 9:56:35,Ting I Yun,國立清華大學,人文社會學系,"""如果你對人文社會性別等主題或是創作有興趣\n卻不知道自己該鑽研哪個部分\n這裡算是個讓你有...",如果你對人文社會性別等主題或是創作有興趣卻不知道自己該鑽研哪個部分這裡算是個讓你有額外的時間...,"[如果, 你, 對, 人文, 社會, 性別, 等, 主題, 或是, 創作, 有, 興趣, 卻...",一般大學


In [40]:
from collections import Counter

school_counts = Counter(df['學校類別'])
school_counts

Counter({'一般大學': 6, '科技大學': 4})

In [41]:
import plotly.express as px

# Sorting the school counts in descending order
sorted_school_counts = sorted(school_counts.items(), key=lambda x: x[1], reverse=True)
sorted_school_counts_dict = dict(sorted_school_counts)

# Creating a sorted DataFrame for the bar chart
sorted_school_df = pd.DataFrame(list(sorted_school_counts_dict.items()), columns=["學校類別", "Count"])

# Plotting the sorted school distribution
fig_sorted = px.bar(sorted_school_df, x='學校類別', y='Count',
                    title="School Distribution (Sorted by Count)",
                    labels={"學校類別": "學校類別", "Count": "Number of Mentions"},
                    text='Count')

fig_sorted.update_traces(texttemplate='%{text}', textposition='outside')
fig_sorted.update_layout(xaxis=dict(title="School"), yaxis=dict(title="Number of Mentions"))

fig_sorted.show()

In [52]:
from google.colab import userdata
apikey = userdata.get('GOOGLE_API_KEY')

In [58]:
import google.generativeai as genai
genai.configure(api_key = apikey)
model = genai.GenerativeModel("gemini-1.5-flash")

prompt = f"從系所欄位判斷其屬於甚麼學院:文學院、理學院(數理化學類)、工程學院(與電腦相關)、藝術學院、社會學院、管理學院(餐飲)、經濟學院，系所欄位以外的其他訊息不計，並務必用csv格式寫出(只需列出XX學院，且不必列出標題)：{df.to_string()}"

response = model.generate_content(prompt)
print(response.text)

```csv
文學院
藝術學院
文學院
文學院
社會學院
科技與工程學院
科技與工程學院
理學院
藝術學院
工程學院
```



In [59]:
from io import StringIO

# 轉換為 DataFrame
df_all = pd.read_csv(StringIO(response.text))
df['學院類別'] = df_all
df

,時間戳記,姓名,校名,系所,留言,Chinese_Content,Tokenized,學校類別,學院類別
0,2025/4/24 上午 9:51:57,曾凱筠,東吳大學,哲學系,東吳哲學 學費+住宿接近五萬\n哲學不是文科其實你要有部分的理科邏輯基礎才有辦法一開始就適應...,東吳哲學學費住宿接近五萬哲學不是文科其實你要有部分的理科邏輯基礎才有辦法一開始就適應我們學到...,"[東吳, 哲學學費, 住宿, 接近, 五萬, 哲學, 不是, 文科, 其實, 你, 要, 有...",一般大學,文學院
1,2025/4/24 上午 9:53:51,Yoou Boonsin,真理大學,台灣文學系,真理台灣文學(後段私校有人選ㄇ)\n地理位置在新北市淡水區，公車班次沒有淡江來得多，接近鄰校...,真理台灣文學後段私校有人選地理位置在新北市淡水區公車班次沒有淡江來得多接近鄰校走路就能到淡江...,"[真理, 台灣, 文學後段, 私校, 有人, 選, 地理位置, 在, 新北市, 淡水, 區公...",一般大學,藝術學院
2,2025/4/24 上午 9:55:39,微波小莫,元智大學,藝術及設計學系,"""學費+學雜沒記錯也快5萬\n師資不差 老師都蠻厲害的\n但我個人認為要很有興趣再考慮會比較...",學費學雜沒記錯也快萬師資不差老師都蠻厲害的但我個人認為要很有興趣再考慮會比較好常常做大型裝置...,"[學費, 學雜, 沒, 記錯, 也, 快, 萬師, 資不差, 老師, 都, 蠻, 厲害, 的...",一般大學,文學院
3,2025/4/24 上午 9:56:08,李豫慈,國立政治大學,歐洲語文學系德文組,"""政大位於台北市的邊陲，學校唯一一班直達市政府的公車每個星期五下午都是塞爆的狀態（是人擠到門...",政大位於台北市的邊陲學校唯一一班直達市政府的公車每個星期五下午都是塞爆的狀態是人擠到門邊的那...,"[政, 大位, 於, 台北市, 的, 邊陲, 學校, 唯一, 一班, 直達, 市政府, 的,...",一般大學,文學院
4,2025/4/24 上午 9:56:35,Ting I Yun,國立清華大學,人文社會學系,"""如果你對人文社會性別等主題或是創作有興趣\n卻不知道自己該鑽研哪個部分\n這裡算是個讓你有...",如果你對人文社會性別等主題或是創作有興趣卻不知道自己該鑽研哪個部分這裡算是個讓你有額外的時間...,"[如果, 你, 對, 人文, 社會, 性別, 等, 主題, 或是, 創作, 有, 興趣, 卻...",一般大學,社會學院
5,2025/4/24 上午 9:57:12,法金,台南應用科技大學,多媒體動畫系,"""雖然身為大一還沒太多經歷\n但就我所能的說（乾）\n/\n台南應用科技大學 多媒體動畫系\...",雖然身為大一還沒太多經歷但就我所能的說乾台南應用科技大學多媒體動畫系一學期學雜費萬地理位置很...,"[雖然, 身為, 大一還, 沒太多, 經歷, 但, 就, 我, 所, 能, 的, 說, 乾,...",科技大學,科技與工程學院
6,2025/4/24 上午 9:58:07,莫西多,弘光科技大學,化妝品應用系,"""學費部分我記得是繳差不多5萬多\n系上內容的話，有幾門課是我很頭痛的😂\n其中一門就是生理...",學費部分我記得是繳差不多萬多系上內容的話有幾門課是我很頭痛的其中一門就是生理學學的就是生物但...,"[學費, 部分, 我, 記得, 是繳, 差不多, 萬多系, 上, 內容, 的, 話, 有, ...",科技大學,科技與工程學院
7,2025/4/24 上午 9:59:15,林影,國立屏東科技大學,餐旅系,"""學費第一期會比較多衣服那些的大概3.4萬，之後都是2萬多，系學會費現在好像是3還是4千一次...",學費第一期會比較多衣服那些的大概萬之後都是萬多系學會費現在好像是還是千一次繳四年用學生會費繳...,"[學費, 第一期, 會, 比, 較, 多, 衣服, 那些, 的, 大概, 萬之後都, 是, ...",科技大學,理學院
8,2025/4/24 上午 9:59:46,夏川蛍,國立台南藝術大學,材質創作與設計系,"""學雜+宿舍大約3萬，宿舍看住哪個宿舍價錢不一樣（房型也會影響，但我住的雙人房有衛浴一個人也...",學雜宿舍大約萬宿舍看住哪個宿舍價錢不一樣房型也會影響但我住的雙人房有衛浴一個人也才快五千系費...,"[學雜, 宿舍, 大約, 萬, 宿舍, 看住, 哪個, 宿舍, 價錢, 不, 一樣, 房型,...",科技大學,藝術學院
9,2025/4/24 上午 10:01:04,曾冠富,國立中正大學,電機學系,"""看到別人分享這串 於是跑來這裡留言\n中正電機\n學費就一般公立那樣 具體多少忘記 三萬內...",看到別人分享這串於是跑來這裡留言中正電機學費就一般公立那樣具體多少忘記三萬內請要上理工相關的...,"[看到, 別人, 分享, 這串, 於, 是, 跑, 來, 這裡, 留言, 中正, 電機, 學...",一般大學,工程學院


In [60]:
# 計算每個 Account 的留言次數
account_message_count = df.groupby('姓名').size().reset_index(name='Message_Count')

# 合併學校名稱到 account_message_count
account_school = df[['姓名', '學院類別']].drop_duplicates()

# 合併留言次數與學校名稱
result = account_message_count.merge(account_school, on='姓名')

# Group the data by 'School' and aggregate the message count and accounts
grouped_data = result.groupby('學院類別').agg({
    '姓名': lambda accounts: ', '.join(map(str, accounts)),  # Convert integers to strings before joining
    'Message_Count': 'sum'
}).reset_index()

# Rename columns for clarity
grouped_data.rename(columns={
    '學院類別': '學院',
    '姓名': '就讀者',
    'Message_Count': '數量'
}, inplace=True)

# Sort the grouped data by Total_Messages in descending order
sorted_data = grouped_data.sort_values(by='數量', ascending=False).reset_index(drop=True)
sorted_data

,學院,就讀者,數量
0,文學院,"微波小莫, 曾凱筠, 李豫慈",3
1,藝術學院,"Yoou Boonsin, 夏川蛍",2
2,科技與工程學院,"法金, 莫西多",2
3,工程學院,曾冠富,1
4,社會學院,Ting I Yun,1
5,理學院,林影,1


In [63]:
import google.generativeai as genai
genai.configure(api_key = apikey)
model = genai.GenerativeModel("gemini-1.5-flash")

prompt = f"從留言10間學校系所分析其學費大約要多少萬，並用csv格式寫出(只需列出幾萬(例如XX萬，若無法判斷及寫無法判斷)，且不必列出標題)：{df.to_string()}"

response = model.generate_content(prompt)
print(response.text)

```csv
五萬
無法判斷
五萬
無法判斷
無法判斷
五萬
五萬
三萬
三萬
三萬
```



In [64]:
from io import StringIO

# 轉換為 DataFrame
df_all = pd.read_csv(StringIO(response.text))
df['學費預估'] = df_all
df

,時間戳記,姓名,校名,系所,留言,Chinese_Content,Tokenized,學校類別,學院類別,學費預估
0,2025/4/24 上午 9:51:57,曾凱筠,東吳大學,哲學系,東吳哲學 學費+住宿接近五萬\n哲學不是文科其實你要有部分的理科邏輯基礎才有辦法一開始就適應...,東吳哲學學費住宿接近五萬哲學不是文科其實你要有部分的理科邏輯基礎才有辦法一開始就適應我們學到...,"[東吳, 哲學學費, 住宿, 接近, 五萬, 哲學, 不是, 文科, 其實, 你, 要, 有...",一般大學,文學院,五萬
1,2025/4/24 上午 9:53:51,Yoou Boonsin,真理大學,台灣文學系,真理台灣文學(後段私校有人選ㄇ)\n地理位置在新北市淡水區，公車班次沒有淡江來得多，接近鄰校...,真理台灣文學後段私校有人選地理位置在新北市淡水區公車班次沒有淡江來得多接近鄰校走路就能到淡江...,"[真理, 台灣, 文學後段, 私校, 有人, 選, 地理位置, 在, 新北市, 淡水, 區公...",一般大學,藝術學院,無法判斷
2,2025/4/24 上午 9:55:39,微波小莫,元智大學,藝術及設計學系,"""學費+學雜沒記錯也快5萬\n師資不差 老師都蠻厲害的\n但我個人認為要很有興趣再考慮會比較...",學費學雜沒記錯也快萬師資不差老師都蠻厲害的但我個人認為要很有興趣再考慮會比較好常常做大型裝置...,"[學費, 學雜, 沒, 記錯, 也, 快, 萬師, 資不差, 老師, 都, 蠻, 厲害, 的...",一般大學,文學院,五萬
3,2025/4/24 上午 9:56:08,李豫慈,國立政治大學,歐洲語文學系德文組,"""政大位於台北市的邊陲，學校唯一一班直達市政府的公車每個星期五下午都是塞爆的狀態（是人擠到門...",政大位於台北市的邊陲學校唯一一班直達市政府的公車每個星期五下午都是塞爆的狀態是人擠到門邊的那...,"[政, 大位, 於, 台北市, 的, 邊陲, 學校, 唯一, 一班, 直達, 市政府, 的,...",一般大學,文學院,無法判斷
4,2025/4/24 上午 9:56:35,Ting I Yun,國立清華大學,人文社會學系,"""如果你對人文社會性別等主題或是創作有興趣\n卻不知道自己該鑽研哪個部分\n這裡算是個讓你有...",如果你對人文社會性別等主題或是創作有興趣卻不知道自己該鑽研哪個部分這裡算是個讓你有額外的時間...,"[如果, 你, 對, 人文, 社會, 性別, 等, 主題, 或是, 創作, 有, 興趣, 卻...",一般大學,社會學院,無法判斷
5,2025/4/24 上午 9:57:12,法金,台南應用科技大學,多媒體動畫系,"""雖然身為大一還沒太多經歷\n但就我所能的說（乾）\n/\n台南應用科技大學 多媒體動畫系\...",雖然身為大一還沒太多經歷但就我所能的說乾台南應用科技大學多媒體動畫系一學期學雜費萬地理位置很...,"[雖然, 身為, 大一還, 沒太多, 經歷, 但, 就, 我, 所, 能, 的, 說, 乾,...",科技大學,科技與工程學院,五萬
6,2025/4/24 上午 9:58:07,莫西多,弘光科技大學,化妝品應用系,"""學費部分我記得是繳差不多5萬多\n系上內容的話，有幾門課是我很頭痛的😂\n其中一門就是生理...",學費部分我記得是繳差不多萬多系上內容的話有幾門課是我很頭痛的其中一門就是生理學學的就是生物但...,"[學費, 部分, 我, 記得, 是繳, 差不多, 萬多系, 上, 內容, 的, 話, 有, ...",科技大學,科技與工程學院,五萬
7,2025/4/24 上午 9:59:15,林影,國立屏東科技大學,餐旅系,"""學費第一期會比較多衣服那些的大概3.4萬，之後都是2萬多，系學會費現在好像是3還是4千一次...",學費第一期會比較多衣服那些的大概萬之後都是萬多系學會費現在好像是還是千一次繳四年用學生會費繳...,"[學費, 第一期, 會, 比, 較, 多, 衣服, 那些, 的, 大概, 萬之後都, 是, ...",科技大學,理學院,三萬
8,2025/4/24 上午 9:59:46,夏川蛍,國立台南藝術大學,材質創作與設計系,"""學雜+宿舍大約3萬，宿舍看住哪個宿舍價錢不一樣（房型也會影響，但我住的雙人房有衛浴一個人也...",學雜宿舍大約萬宿舍看住哪個宿舍價錢不一樣房型也會影響但我住的雙人房有衛浴一個人也才快五千系費...,"[學雜, 宿舍, 大約, 萬, 宿舍, 看住, 哪個, 宿舍, 價錢, 不, 一樣, 房型,...",科技大學,藝術學院,三萬
9,2025/4/24 上午 10:01:04,曾冠富,國立中正大學,電機學系,"""看到別人分享這串 於是跑來這裡留言\n中正電機\n學費就一般公立那樣 具體多少忘記 三萬內...",看到別人分享這串於是跑來這裡留言中正電機學費就一般公立那樣具體多少忘記三萬內請要上理工相關的...,"[看到, 別人, 分享, 這串, 於, 是, 跑, 來, 這裡, 留言, 中正, 電機, 學...",一般大學,工程學院,三萬


In [66]:
pip install plotly

In [68]:
import pandas as pd
import plotly.express as px

# 假設你已經有 df
# 把中文的學費欄位轉成數值
mapping = {
    '三萬': 30000,
    '五萬': 50000,
    '無法判斷': None
}
df['學費預估數值'] = df['學費預估'].map(mapping)

# 製作透視表：index=學校類別, columns=學院類別, values=平均學費
heatmap_data = df.pivot_table(
    index='學校類別',
    columns='學院類別',
    values='學費預估數值',
    aggfunc='mean'
)

# 重設索引讓 Plotly 好操作
heatmap_data = heatmap_data.reset_index().melt(id_vars='學校類別', var_name='學院類別', value_name='平均學費')

# 畫互動式熱力圖
fig = px.density_heatmap(
    heatmap_data,
    x='學院類別',
    y='學校類別',
    z='平均學費',
    color_continuous_scale='YlOrRd',
    text_auto=True,
    title="學校類別 × 學院類別 的平均學費熱力圖(0代表缺乏數據)"
)

fig.update_layout(xaxis_title='學院類別', yaxis_title='學校類別')
fig.show()

In [ ]:
import google.generativeai as genai
genai.configure(api_key = apikey)
model = genai.GenerativeModel("gemini-1.5-flash")

prompt = f"用繁體中文綜合分析各科系，不必列出原表格：{df.to_string()}"

response = model.generate_content(prompt)
print(response.text)

綜合以上各科系留言，可以看出學生們分享的內容涵蓋學業、生活、環境等面向，呈現多元且立體的科系樣貌。

**學業方面：**  哲學系著重批判性思考與多元觀點的培養，需具備一定的邏輯基礎；台灣文學系則以台灣文學史為主軸，較少著重傳統中國文學；藝術與設計學系強調實作能力，需投入大量時間進行大型作品創作，並需具備一定的電腦繪圖和程式設計能力；歐洲語文學系德文組則偏重基礎文法教學，口語和聽力能力則需仰賴自學；人文社會學系涵蓋範圍廣泛，適合對人文社會議題有興趣但尚未確定方向的學生；多媒體動畫系著重動畫和3D製作，需具備一定的藝術天賦和軟體操作能力；化妝品應用系則分美技組和科技組，前者著重實務操作，後者則需具備一定的化學和理科知識；餐旅系課程多元，涵蓋實務技能和理論知識，並提供技能選手培訓；材質創作與設計系則需要投入大量的時間和金錢成本於創作，並需具備一定的自我規劃能力；電機系則強調數理基礎的重要性，環環相扣的課程內容需要學生打好基礎。

**生活與環境方面：**  不同學校的地理位置、交通便利性、住宿條件、生活費用等都有所差異。例如，部分學校位於偏遠地區或交通不便之處，學生可能需要自行解決交通問題；部分學校周邊生活機能較差，飲食選擇較少或價格較高；宿舍條件也各有不同，部分學校宿舍中籤率低，學生可能需要考慮外宿。另外，部分科系需要自行負擔額外的材料費或比賽費用等。

**整體而言：**  學生們的分享展現了大學生活多樣的面向，提醒未來學生在選擇科系時，除了考慮自身興趣和能力外，也應充分了解各科系的課程內容、學習環境、生活成本等因素，才能做出最適合自己的選擇。 選擇科系前應仔細評估自身能力與興趣，並做好充分的心理準備，才能在大學生活中獲得滿意的學習體驗。  部分科系對學生的自主學習能力和時間管理能力要求較高，學生需要做好時間規劃，並積極主動地學習和探索。  最後，許多學生提到師資和同學對學習的影響，顯示良好的人際關係和師生互動對大學生活的重要性。

